In [1]:
import gensim
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
#from gensim import corpora, models
import pyLDAvis.gensim
import os

df = pd.read_csv('restaurant_cleaned.csv')

df.columns

C:\Users\Aster\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Index(['Unnamed: 0', 'review_id', 'user_id', 'business_id', 'stars_x', 'date',
       'text', 'useful', 'funny', 'cool', 'name', 'neighborhood', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars_y',
       'review_count', 'is_open', 'categories', 'lang'],
      dtype='object')

In [2]:
docs=df['text'].apply(word_tokenize)

In [3]:
d= gensim.corpora.Dictionary(docs)

#convert docs to vec
v = [d.doc2bow(doc) for doc in docs]

#tfidf = models.TfidfModel(v, normalize=True)
#v = tfidf[v] #tfidf score

In [4]:
os.environ.update({'MALLET_HOME':r'D:/mallet-2.0.8/'})

mallet_path = r'D:\mallet-2.0.8\bin\mallet'

ldamallet= gensim.models.wrappers.LdaMallet(mallet_path, corpus = v, num_topics = 8, id2word = d)

vecTop= ldamallet.show_topics()
for i in range(0, 8):
    print(vecTop[i])

C:\Users\Aster\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(0, '0.027*"tabl" + 0.023*"time" + 0.021*"order" + 0.017*"wait" + 0.016*"servic" + 0.015*"minut" + 0.014*"server" + 0.014*"busi" + 0.013*"back" + 0.013*"peopl"')
(1, '0.022*"dish" + 0.017*"time" + 0.016*"love" + 0.016*"staff" + 0.015*"atmospher" + 0.015*"recommend" + 0.014*"drink" + 0.013*"experi" + 0.013*"japanes" + 0.013*"restaur"')
(2, '0.035*"order" + 0.017*"restaur" + 0.016*"nice" + 0.016*"portion" + 0.015*"tast" + 0.013*"small" + 0.013*"menu" + 0.013*"3" + 0.012*"2" + 0.011*"servic"')
(3, '0.031*"burger" + 0.023*"fri" + 0.018*"eat" + 0.017*"chees" + 0.017*"make" + 0.016*"time" + 0.013*"option" + 0.011*"thing" + 0.011*"menu" + 0.010*"review"')
(4, '0.059*"sandwich" + 0.047*"smoke" + 0.030*"line" + 0.024*"montreal" + 0.021*"wait" + 0.017*"pickl" + 0.016*"schwartz" + 0.015*"worth" + 0.013*"visit" + 0.012*"long"')
(5, '0.037*"fri" + 0.026*"pork" + 0.022*"mi" + 0.019*"beef" + 0.018*"banh" + 0.016*"delici" + 0.015*"kimchi" + 0.014*"belli" + 0.012*"bao" + 0.011*"love"')
(6, '0.016*"pout

In [12]:
import joblib
#Save a model to disk, or reload a pre-trained model

joblib.dump(ldamallet, 'reviews_ldamallet.jl')

# # then reload it with
ldamallet_disk = joblib.load('reviews_ldamallet.jl')

In [13]:
vecTop= ldamallet_disk.show_topics()
for i in range(0, 8):
    print(vecTop[i])

(0, '0.027*"tabl" + 0.023*"time" + 0.021*"order" + 0.017*"wait" + 0.016*"servic" + 0.015*"minut" + 0.014*"server" + 0.014*"busi" + 0.013*"back" + 0.013*"peopl"')
(1, '0.022*"dish" + 0.017*"time" + 0.016*"love" + 0.016*"staff" + 0.015*"atmospher" + 0.015*"recommend" + 0.014*"drink" + 0.013*"experi" + 0.013*"japanes" + 0.013*"restaur"')
(2, '0.035*"order" + 0.017*"restaur" + 0.016*"nice" + 0.016*"portion" + 0.015*"tast" + 0.013*"small" + 0.013*"menu" + 0.013*"3" + 0.012*"2" + 0.011*"servic"')
(3, '0.031*"burger" + 0.023*"fri" + 0.018*"eat" + 0.017*"chees" + 0.017*"make" + 0.016*"time" + 0.013*"option" + 0.011*"thing" + 0.011*"menu" + 0.010*"review"')
(4, '0.059*"sandwich" + 0.047*"smoke" + 0.030*"line" + 0.024*"montreal" + 0.021*"wait" + 0.017*"pickl" + 0.016*"schwartz" + 0.015*"worth" + 0.013*"visit" + 0.012*"long"')
(5, '0.037*"fri" + 0.026*"pork" + 0.022*"mi" + 0.019*"beef" + 0.018*"banh" + 0.016*"delici" + 0.015*"kimchi" + 0.014*"belli" + 0.012*"bao" + 0.011*"love"')
(6, '0.016*"pout

In [5]:

#put the reviews data to the model, v are the reviews after doc2bow
m = ldamallet[v[0:len(v)]]

#choose the topic with highest ratio 
import operator 
topic = []
for x in m:
    t = max(x, key = operator.itemgetter(1))
    topic.append(t[0])
    

In [6]:
#sentiment analysis 

top_10k = pd.read_csv('restaurant_cleaned.csv')

pos_lexicon = 'positive-words.txt'
neg_lexicon = 'negative-words.txt'

top_10k = top_10k.loc[:, ['name','business_id', 'text']]

top_10k['text_sep'] = top_10k['text'].map(lambda x:x.split())


#import positive dictionary
pos_dict = {}
f = open(pos_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    pos_dict[line] = 1
        
f.close()

#trim the dictionary to comply with the stemmed word: amaz, simpl


#import negative dictionary
neg_dict = {}
f = open(neg_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    neg_dict[line] = 1
        
f.close()



#calculate the positive score of each review
score = top_10k['text_sep'].map(lambda row: list(map(lambda x: 1 if x in pos_dict else 0, row)))
score_list = []

for row in score:
    score_list.append(sum(row))
score_df = pd.DataFrame(score_list)
top_10k['pos_score'] = score_df


#calculate the negative score of each review

score = top_10k['text_sep'].map(lambda row: list(map(lambda x: -1 if x in neg_dict else 0, row)))
score_list = []

for row in score:
    score_list.append(sum(row))
score_df = pd.DataFrame(score_list)
top_10k['neg_score'] = score_df


top_10k['total_score'] = top_10k['pos_score'] + top_10k['neg_score']



top_10k['topic'] = list(topic)
top_10k



,name,business_id,text,text_sep,pos_score,neg_score,total_score,topic
0,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,super simpl amaz nonetheless around sinc 30 st...,"[super, simpl, amaz, nonetheless, around, sinc...",1,0,1,4
1,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,shop featur mind chef anthoni bourdain david c...,"[shop, featur, mind, chef, anthoni, bourdain, ...",1,-1,0,4
2,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,name suggest wilenski lunch counteron that ope...,"[name, suggest, wilenski, lunch, counteron, th...",2,0,2,4
3,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,stop wilenski special love almost shed tear ta...,"[stop, wilenski, special, love, almost, shed, ...",3,0,3,4
4,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,admit fan enter know time stop mani mani year ...,"[admit, fan, enter, know, time, stop, mani, ma...",1,-1,0,4
5,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,went visit montreal simpl yet delici special c...,"[went, visit, montreal, simpl, yet, delici, sp...",0,-2,-2,4
6,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,disgust compon hotdog handl three separ peopl ...,"[disgust, compon, hotdog, handl, three, separ,...",0,-6,-6,0
7,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,70 year make signatur sandwich way take serv s...,"[70, year, make, signatur, sandwich, way, take...",4,0,4,3
8,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,hear mani thing fame variou critic person fina...,"[hear, mani, thing, fame, variou, critic, pers...",7,-8,-1,4
9,"""Wilensky's""",AEx2SYEUJmTxVVB18LlCwA,wilenski import remind montreal heritag wilens...,"[wilenski, import, remind, montreal, heritag, ...",1,0,1,4


In [7]:
#map topic name back to the topic numbers
topic_dict = {0:'Atmosphere', 1:'Food', 2:'Service',
             3:'Food', 4:'Waiting time', 5:'Food',
             6:'Food', 7:'Hospitality'}

top_10k = top_10k.replace({'topic':topic_dict})

In [8]:
top_10k['topic_score'] = top_10k['pos_score'] / (top_10k['pos_score'] + abs(top_10k['neg_score']) + 1 )*5



top_10k_score = top_10k.iloc[:, [0, 7, 8]]

total_score = top_10k_score.groupby(['name', 'topic']).sum()
total_num_review = top_10k_score.groupby(['name', 'topic']).count()

final_score = round(total_score/total_num_review, 1)

In [9]:
print(final_score)

                                                topic_score
name                              topic                    
"Banh Mi Boys"                    Atmosphere            2.0
                                  Food                  2.6
                                  Hospitality           2.3
                                  Service               2.7
                                  Waiting time          2.0
"Basil Thai Kitchen"              Atmosphere            2.5
                                  Food                  2.3
                                  Hospitality           1.7
                                  Service               2.6
"Black Hoof"                      Atmosphere            2.2
                                  Food                  2.9
                                  Hospitality           2.1
                                  Service               2.3
                                  Waiting time          1.7
"Bonjour Brioche"                 Atmosp

In [14]:
top_10k.to_csv("top_10k_mallet.csv")